In [2]:
import pandas as pd
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama 
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain 

In [3]:

df = pd.read_csv("cards.csv")
loader = PyPDFLoader("meanings.pdf")
docs = loader.load()


In [4]:
## transformation by splitting
text_splitters = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap =200)
documents = text_splitters.split_documents(docs)


In [5]:
## embedding the documents as vector

db = FAISS.from_documents(documents,OllamaEmbeddings())

In [6]:
## initializing llm model
llm = Ollama(model="llama2")

## designing chat prompt template

prompt = ChatPromptTemplate.from_template("""
You are an experienced tarot reader. You derive meaningful interpretations
from the cards drawn and guide the querent with kindness, honesty and empathy . You 
refer to the symbolisms and mythology to empower the querent.
<context>{context}</context>
Question:{input}
"""
)


In [7]:
document_chain = create_stuff_documents_chain(llm,prompt)

In [8]:
## initializing retriver

retriever= db.as_retriever()

## creating retrival chain 
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [9]:
def get_reading(card1,card2,card3,question):
    cards_drawn = df.at[card1, 'Name']+ ", " +df.at[card2, 'Name']+ ", " + df.at[card3, 'Name']
    response = retrieval_chain.invoke({"context": cards_drawn,
                                       "question": question 
                                        })
    return response['answer']



In [1]:
print(get_reading(0,10,56,"how can i improve my relationships?"))
